In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, concat_ws, regexp_replace, trim


https://drive.google.com/drive/folders/18Fnrb5fwJNXxJhUarmRn9-LZlbrbpsOm?usp=drive_link

In [2]:
#Instalação das bibliotecas necessárias para o projeto
!pip install pyspark pandas pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [3]:

# Conexão com o Spark

spark = (
    SparkSession.builder
        .appName("LinkedInJobsAnalysis")
        .master("local[1]")              # <= limita workers a 1 threads
        .config("spark.sql.shuffle.partitions", "8")
        .config("spark.port.maxRetries", "128")
        .config("spark.python.worker.reuse", "true")
        .config("spark.python.connect.timeout", "180")
        .getOrCreate()
)

# logar URL do Spark UI
print("Spark UI:", spark.sparkContext.uiWebUrl)


Spark UI: http://DESKTOP-MC695QT:4040


In [4]:
#Leitura do arquivo CSV com spark
sdf = spark.read.csv("Linkedin_dataset\postings.csv", header=True,sep=",",quote='"', escape='"',multiLine=True,inferSchema=False)
sdf.printSchema()
sdf.show(5, truncate=False)

root
 |-- job_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- max_salary: string (nullable = true)
 |-- pay_period: string (nullable = true)
 |-- location: string (nullable = true)
 |-- company_id: string (nullable = true)
 |-- views: string (nullable = true)
 |-- med_salary: string (nullable = true)
 |-- min_salary: string (nullable = true)
 |-- formatted_work_type: string (nullable = true)
 |-- applies: string (nullable = true)
 |-- original_listed_time: string (nullable = true)
 |-- remote_allowed: string (nullable = true)
 |-- job_posting_url: string (nullable = true)
 |-- application_url: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- expiry: string (nullable = true)
 |-- closed_time: string (nullable = true)
 |-- formatted_experience_level: string (nullable = true)
 |-- skills_desc: string (nullable = true)
 |-- listed_time: string (nullable

In [5]:
#Estatísticas descritivas e básicas do spark dataframe
sdf.describe().show()

+-------+-------------------+----------------+--------------------+--------------------+-----------------+----------+--------------+--------------------+------------------+------------------+-----------------+-------------------+------------------+--------------------+--------------+--------------------+------------------+------------------+--------------------+--------------------+--------------------------+--------------------+--------------------+------------------+---------+---------+--------+-----------------+------------------+-----------------+------------------+
|summary|             job_id|    company_name|               title|         description|       max_salary|pay_period|      location|          company_id|             views|        med_salary|       min_salary|formatted_work_type|           applies|original_listed_time|remote_allowed|     job_posting_url|   application_url|  application_type|              expiry|         closed_time|formatted_experience_level|         sk

In [6]:
#Correção de tipos de dados
# A função "cast" é utilizada para converter os tipos de dados das colunas do DataFrame.
df = (
    sdf
    .withColumn("min_salary", col("min_salary").cast("double"))
    .withColumn("med_salary", col("med_salary").cast("double"))
    .withColumn("max_salary", col("max_salary").cast("double"))
    .withColumn("normalized_salary", col("normalized_salary").cast("double"))
    .withColumn("views", col("views").cast("long"))
    .withColumn("applies", col("applies").cast("long"))
    .withColumn("company_id", col("company_id").cast("long"))
    .withColumn("listed_time",       (col("listed_time").cast("double")/1000).cast("timestamp"))
    .withColumn("original_listed_time",(col("original_listed_time").cast("double")/1000).cast("timestamp"))
    .withColumn("closed_time",       (col("closed_time").cast("double")/1000).cast("timestamp"))
    .withColumn("expiry",            (col("expiry").cast("double")/1000).cast("timestamp"))
)

df.printSchema()

root
 |-- job_id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- max_salary: double (nullable = true)
 |-- pay_period: string (nullable = true)
 |-- location: string (nullable = true)
 |-- company_id: long (nullable = true)
 |-- views: long (nullable = true)
 |-- med_salary: double (nullable = true)
 |-- min_salary: double (nullable = true)
 |-- formatted_work_type: string (nullable = true)
 |-- applies: long (nullable = true)
 |-- original_listed_time: timestamp (nullable = true)
 |-- remote_allowed: string (nullable = true)
 |-- job_posting_url: string (nullable = true)
 |-- application_url: string (nullable = true)
 |-- application_type: string (nullable = true)
 |-- expiry: timestamp (nullable = true)
 |-- closed_time: timestamp (nullable = true)
 |-- formatted_experience_level: string (nullable = true)
 |-- skills_desc: string (nullable = true)
 |-- listed_time: timestamp (nu

In [7]:
df.show(truncate=False)

+---------+---------------------------------------+--------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [8]:
#Seleção de colunas mais importantes para análise
df.select("min_salary","med_salary","max_salary","normalized_salary","applies","views","remote_allowed", "skills_desc").summary().show(truncate=False)

+-------+-----------------+------------------+-----------------+------------------+------------------+------------------+--------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|summary|min_salary       |med_salary        |max_salary       |normalized_salary |applies           |views             |remote_allowed|skills_desc                                                                                                                                                                                                                                        

In [9]:
#Filtragem de dados nulos

df.filter(col("min_salary").isNull()).count()
df.filter(col("med_salary").isNull()).count()   
df.filter(col("max_salary").isNull()).count() 
df.filter(col("normalized_salary").isNull()).count() 

87776

In [10]:
from pyspark.sql import functions as F

# 1) Catálogo das tecnologias usadas em dados 
TECHS = [
    "python", "r", "sql", "bigquery",
"pandas", "numpy", "scikit-learn", "tensorflow", "pytorch", "keras",
    "spark", "hadoop", "power bi", "streamlit",
]

# 2) Constrói array literal das techs
tech_array = F.array(*[F.lit(t) for t in TECHS])

# 3) Cria coluna "words" e, em seguida, "tech_list"
df = (
    df
    # remove tudo que não for letra, número, + ou #, substitui por espaço
    .withColumn(
        "words",
        F.split(
            F.lower(
                F.regexp_replace(F.col("description"), "[^A-Za-z0-9+#]", " ")
            ),
            "\\s+"
        )
    )
    # interseção entre palavras e tech_array, sem duplicatas
    .withColumn(
        "tech_list",
        F.array_distinct(
            F.array_intersect(F.col("words"), tech_array)
        )
    )
    # opcional: remover a coluna auxiliar
    .drop("words")
)

# 4) Filtro para visualizar as vagas de dados,apenas com as colunas de interesse e filtrar para apenas trabalho remoto
novo_df = (df
 .filter(F.size("tech_list") > 4)
 .select("company_name","title", "tech_list","min_salary","med_salary","max_salary","normalized_salary","applies","views","remote_allowed", "skills_desc","formatted_experience_level")
 .show(100, truncate=False)
)




+--------------------------------------------------------+----------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+----------+----------+----------+-----------------+-------+-----+--------------+-----------+--------------------------+
|company_name                                            |title                                                                                               |tech_list                                                              |min_salary|med_salary|max_salary|normalized_salary|applies|views|remote_allowed|skills_desc|formatted_experience_level|
+--------------------------------------------------------+----------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------+----------+----------+----------+-----------------+-------+-----+--

In [11]:
from pyspark.sql import functions as F

# 1) Catálogo das tecnologias 
TECHS = [
    "python", "r", "sql", "bigquery",
    "pandas", "numpy", "scikit-learn", "tensorflow", "pytorch", "keras",
    "spark", "hadoop", "power bi", "streamlit",
]

# 2) Array literal das techs
tech_array = F.array(*[F.lit(t) for t in TECHS])

# interseção entre palavras e tech_array, sem duplicatas
df = (
    df
    .withColumn(
        "words",
        F.split(
            F.lower(F.regexp_replace("description", "[^A-Za-z0-9+#]", " ")),
            "\\s+"
        )
    )
    .withColumn(
        "tech_list",
        F.array_distinct(F.array_intersect("words", tech_array))
    )
    .drop("words")
)

# 4) Filtro para visualizar as vagas de dados,apenas com as colunas de interesse e filtrar para apenas trabalho remoto
novo_df = (
    df
    .filter( F.size("tech_list") > 4 )                          
    .filter(col("remote_allowed") == 1.0) # Filtra apenas as vagas remotas          
    .select(
        "company_name",
        "title",
        "tech_list",
        "min_salary", "med_salary", "max_salary", "normalized_salary",
        "applies", "views",
        "remote_allowed",
        "skills_desc",
        "formatted_experience_level"
    )
)

novo_df.show(100, truncate=False)


+----------------------------------+---------------------------------------------------------------------------------+-----------------------------------------------------------------------+----------+----------+----------+-----------------+-------+-----+--------------+-----------+--------------------------+
|company_name                      |title                                                                            |tech_list                                                              |min_salary|med_salary|max_salary|normalized_salary|applies|views|remote_allowed|skills_desc|formatted_experience_level|
+----------------------------------+---------------------------------------------------------------------------------+-----------------------------------------------------------------------+----------+----------+----------+-----------------+-------+-----+--------------+-----------+--------------------------+
|Avesta Computer Services          |Senior Data Engineer (Property Rei

+---------+---------------------------------------+--------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [12]:
from pyspark.sql import functions as F

# 1) Garante que skills_desc não seja nulo e faz o split
novo_df = df.withColumn(
    "skills_list",
    F.split(
        F.coalesce(F.col("skills_desc"), F.lit("")),  # converte null em ""
        r"[;,]\s*"
    )
)

# 2) Trim em cada elemento do array para tirar espaços sobressalentes
novo_df = novo_df.withColumn(
    "skills_list",
    F.expr("transform(skills_list, x -> trim(x))")
)

# 3)filtra só valores não vazios e conta
(
    novo_df
    .select(F.explode("skills_list").alias("skill"))
    .filter(F.col("skill") != "")          # descarta strings vazias
    .groupBy("skill")
    .count()
    .orderBy(F.col("count").desc())
    .show(20, truncate=False)
)



+---------------------------------------------------------------+-----+
|skill                                                          |count|
+---------------------------------------------------------------+-----+
|People Skills                                                  |74   |
|Healthcare                                                     |65   |
|Hospice Care                                                   |60   |
|Patient Care                                                   |56   |
|Fundraising                                                    |53   |
|Verbal / Written Communication                                 |53   |
|religion                                                       |42   |
|CSR / Volunteer Coordination                                   |42   |
|color                                                          |41   |
|national origin                                                |41   |
|sexual orientation                                             

In [13]:
#Filtra informações de total,salario_medio,total_views e Applies_medio por tecnologia
tech_exploded = novo_df.select(
    F.explode("tech_list").alias("tech"),
    "med_salary", "views", "applies"
).filter(F.col("tech") != "")

resumo_tech = (
    tech_exploded.groupBy("tech")
      .agg(
         F.count("*").alias("vagas"),
         F.round(F.avg("med_salary"),2).alias("salario_medio"),
         F.sum("views").alias("total_views"),
         F.round(F.avg("applies"),2).alias("applies_medio")
      )
      .orderBy(F.col("vagas").desc())
)
resumo_tech.show(10)


+----------+-----+-------------+-----------+-------------+
|      tech|vagas|salario_medio|total_views|applies_medio|
+----------+-----+-------------+-----------+-------------+
|         r| 5210|     33071.58|      70547|         9.88|
|       sql| 5181|     61366.82|     183703|        19.27|
|    python| 4647|      55635.9|     139689|        17.34|
|     spark|  856|      78392.0|      26230|         18.7|
|    hadoop|  313|     76943.75|       9832|        15.83|
|   pytorch|  243|     100035.0|       6602|        14.91|
|tensorflow|  224|     100035.0|       5811|         14.8|
|  bigquery|  101|      95000.0|       4825|         22.3|
|    pandas|   95|      90032.5|       3446|        17.65|
|     numpy|   85|      90032.5|       2800|        17.95|
+----------+-----+-------------+-----------+-------------+
only showing top 10 rows



In [14]:
#Filtrando as colunas de interesse para o resultado final
df_final = novo_df.select(
    "company_name","title",
    "min_salary","med_salary","max_salary","normalized_salary",
    "views","applies","tech_list","remote_allowed","formatted_experience_level"
)
df_final.show(50, truncate=False)

+---------------------------------------------------------------------------------+--------------------------------------------------------+----------+----------+----------+-----------------+-----+-------+---------+--------------+--------------------------+
|company_name                                                                     |title                                                   |min_salary|med_salary|max_salary|normalized_salary|views|applies|tech_list|remote_allowed|formatted_experience_level|
+---------------------------------------------------------------------------------+--------------------------------------------------------+----------+----------+----------+-----------------+-----+-------+---------+--------------+--------------------------+
|Corcoran Sawyer Smith                                                            |Marketing Coordinator                                   |17.0      |NULL      |20.0      |38480.0          |20   |2      |[]       |NULL       

In [ ]:
#Convertendo o dataframe do spark para pandas e exportando para CSV
pandas_df = df_final.toPandas()
pandas_df.to_csv("transformed_postings.csv", index=False)


In [ ]:
import pandas as pd
#Leitura do DataFrame
df_pd = pd.read_csv("transformed_postings.csv")

#  Estatísticas numéricas
print(df_pd[["min_salary","med_salary","max_salary","views","applies"]].describe(), "\n")

#  Matriz de correlação
print(df_pd[["med_salary","views","applies"]].corr(), "\n")

# Top 10 tecnologias
top_techs = df_pd.explode("tech_list")["tech_list"].value_counts().head(10)
print("Top 10 Tecnologias:\n", top_techs, "\n")

         min_salary     med_salary    max_salary          views       applies
count  2.979300e+04    6280.000000  2.979300e+04  122160.000000  23320.000000
mean   6.491085e+04   22015.619876  9.193942e+04      14.618247     10.591981
std    4.959738e+05   52255.873846  7.011101e+05      85.903598     29.047395
min    1.000000e+00       0.000000  1.000000e+00       1.000000      1.000000
25%    3.700000e+01      18.940000  4.828000e+01       3.000000      1.000000
50%    6.000000e+04      25.500000  8.000000e+04       4.000000      3.000000
75%    1.000000e+05    2510.500000  1.400000e+05       8.000000      8.000000
max    8.500000e+07  750000.000000  1.200000e+08    9975.000000    967.000000 

            med_salary     views   applies
med_salary    1.000000  0.037125 -0.023131
views         0.037125  1.000000  0.494305
applies      -0.023131  0.494305  1.000000 

Top 10 Tecnologias:
 tech_list
[]                        111351
['r']                       4220
['sql']                  

In [47]:
display(df_pd.head(10))

,company_name,title,min_salary,med_salary,max_salary,normalized_salary,views,applies,tech_list,remote_allowed,formatted_experience_level
0,Corcoran Sawyer Smith,Marketing Coordinator,17.0,NaN,20.0,38480.0,20.0,2.0,[],NaN,NaN
1,NaN,Mental Health Therapist/Counselor,30.0,NaN,50.0,83200.0,1.0,NaN,[],NaN,NaN
2,The National Exemplar,Assitant Restaurant Manager,45000.0,NaN,65000.0,55000.0,8.0,NaN,[],NaN,NaN
3,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,140000.0,NaN,175000.0,157500.0,16.0,NaN,[],NaN,NaN
4,NaN,Service Technician,60000.0,NaN,80000.0,70000.0,3.0,NaN,[],NaN,NaN
5,Downtown Raleigh Alliance,Economic Development and Planning Intern,14.0,NaN,20.0,35360.0,9.0,4.0,[],NaN,NaN
6,Raw Cereal,Producer,60000.0,NaN,300000.0,180000.0,7.0,1.0,[],1.0,NaN
7,NaN,Building Engineer,90000.0,NaN,120000.0,105000.0,2.0,NaN,[],NaN,NaN
8,Children's Nebraska,Respiratory Therapist,NaN,NaN,NaN,NaN,3.0,NaN,[],NaN,NaN
9,Bay West Church,Worship Leader,NaN,350.0,NaN,4200.0,5.0,NaN,[],NaN,NaN
